In [1]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

# RUTAS

### AP JOURNAL

In [2]:
ap_provision_files = glob.glob(r'G:\My Drive\RESPALDO-PC\DYNAMO\2023\02. FEBRERO\PROVISION - LEDGER\FEES TAXES\P*.tsv', recursive=False)
ap_provision_files

['G:\\My Drive\\RESPALDO-PC\\DYNAMO\\2023\\02. FEBRERO\\PROVISION - LEDGER\\FEES TAXES\\PROVISIONES - LEDGER FEBRERO 2023 - FEES TAXES.tsv']

In [3]:
use_cols_approv = ['MERCHANT_ID','MVNT_ID','TYPE_REQUEST','MOVEMENT_TYPE','SOURCE_TYPE','CREATED_AT','EFF_DT','REPORT_DATE_LOCAL',
                  'PAYMENT_DATE_LOCAL','STATUS','AMOUNT']

# DATA AP JOURNAL

In [4]:
ap = pd.concat([pd.read_csv(f, sep='\t',usecols=use_cols_approv).assign(file_name=os.path.basename(f)) for f in ap_provision_files])
ap.columns = ap.columns.str.strip().str.lower()
ap.replace('""','', regex=True, inplace=True)
ap['eff_dt'] =  pd.to_datetime(ap['eff_dt']).dt.tz_localize(None)
ap['eff_dt_date'] =  pd.to_datetime(ap['eff_dt']).dt.date
ap['eff_dt_date'] =  pd.to_datetime(ap['eff_dt_date'])
ap['eff_dt_time'] =  pd.to_datetime(ap['eff_dt']).dt.time
# ap['movement_day'] = pd.to_datetime(ap['movement_day']).dt.tz_localize(None)
ap['created_at'] = pd.to_datetime(ap['created_at']).dt.tz_localize(None)
ap['created_at_date'] =  pd.to_datetime(ap['created_at']).dt.date
ap['created_at_date'] =  pd.to_datetime(ap['created_at_date'])
ap['report_date_local'] =  pd.to_datetime(ap['report_date_local'])
ap['payment_date_local'] = pd.to_datetime(ap['payment_date_local']).dt.tz_localize(None)
ap['payment_date_local_prev'] =  pd.to_datetime(ap['payment_date_local']).dt.date
ap['payment_date_local_prev'] =  pd.to_datetime(ap['payment_date_local_prev'])
# ap['request_effective_date'] = pd.to_datetime(ap['request_effective_date']).dt.tz_localize(None)
# ap['slot_date'] = pd.to_datetime(ap['slot_date']).dt.tz_localize(None)
# ap['identifier'] = ap['source_type'].str[:1].astype(str)
# ap['INDEX'] = (ap['eff_dt_date'].astype(str).replace(' ', '',regex=True)+'-'+
#                ap['identifier'].astype(str).replace(' ', '',regex=True)+'-'+
#                ap['eff_dt_time'].astype(str).replace(' ', '',regex=True))
ap.sort_values(by=(['merchant_id','eff_dt']), ascending=[True,False], inplace=True)

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_4616\2070668920.py:1: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  ap = pd.concat([pd.read_csv(f, sep='\t',usecols=use_cols_approv).assign(file_name=os.path.basename(f)) for f in ap_provision_files])


In [5]:
ap.reset_index(inplace=True)

In [6]:
ap.loc[ap['source_type'] == 'DISB', 'report_id'] = ap['type_request']
ap['report_id'] = ap['report_id'].groupby(ap['merchant_id']).fillna(method='ffill')
ap['report_date_local_final'] = ap['report_date_local'].groupby(ap['merchant_id']).fillna(method='ffill')
ap['payment_date_local_final'] = ap['payment_date_local_prev'].groupby(ap['merchant_id']).fillna(method='ffill')
ap['status_final'] = ap['status'].groupby(ap['merchant_id']).fillna(method='ffill')
ap.head()

,index,merchant_id,amount,type_request,mvnt_id,movement_type,source_type,created_at,eff_dt,report_date_local,...,status,file_name,eff_dt_date,eff_dt_time,created_at_date,payment_date_local_prev,report_id,report_date_local_final,payment_date_local_final,status_final
0,974700,00003751-a43a-4dda-b26d-0e00360ada3a,-670.77,AL87YCZ3P,f2903920-a3da-4c51-9a33-f7c977c06294,DISB_RQ,DISB,2023-03-21 07:13:48,2023-03-21 07:13:53,2023-03-21,...,PAID,PROVISIONES - LEDGER FEBRERO 2023 - FEES TAXES...,2023-03-21,07:13:53,2023-03-21,2023-03-21,AL87YCZ3P,2023-03-21,2023-03-21,PAID
1,10832260,00003751-a43a-4dda-b26d-0e00360ada3a,670.77,settlement processor regular payment request,ecaf79f0-3b4c-4f83-89b3-e11e3bbe05e8,REGULAR_PYMT_RQ,SETT,2023-03-20 17:50:45,2023-03-21 04:30:00,NaT,...,NaN,PROVISIONES - LEDGER FEBRERO 2023 - FEES TAXES...,2023-03-21,04:30:00,2023-03-20,NaT,AL87YCZ3P,2023-03-21,2023-03-21,PAID
2,3092182,00003751-a43a-4dda-b26d-0e00360ada3a,"-1,742.48",A5GGCS72L,b536be83-9b50-4581-ac31-cb68a5244941,DISB_RQ,DISB,2023-03-19 07:16:48,2023-03-19 07:16:53,2023-03-19,...,PAID,PROVISIONES - LEDGER FEBRERO 2023 - FEES TAXES...,2023-03-19,07:16:53,2023-03-19,2023-03-19,A5GGCS72L,2023-03-19,2023-03-19,PAID
3,10832259,00003751-a43a-4dda-b26d-0e00360ada3a,"1,742.48",settlement processor regular payment request,dfb0cce8-4daa-4853-a1b1-6c39719b12be,REGULAR_PYMT_RQ,SETT,2023-03-18 19:27:02,2023-03-19 05:00:00,NaT,...,NaN,PROVISIONES - LEDGER FEBRERO 2023 - FEES TAXES...,2023-03-19,05:00:00,2023-03-18,NaT,A5GGCS72L,2023-03-19,2023-03-19,PAID
4,1817772,00004998-66f3-4c36-9096-4f64f971d19b,-219.44,AFDCWJ6N2,a824e1c2-8d3a-4660-9922-e7168ad72863,DISB_RQ,DISB,2023-03-20 07:15:37,2023-03-20 07:15:42,2023-03-20,...,PAID,PROVISIONES - LEDGER FEBRERO 2023 - FEES TAXES...,2023-03-20,07:15:42,2023-03-20,2023-03-20,AFDCWJ6N2,2023-03-20,2023-03-20,PAID


In [7]:
ap.shape

(12414325, 21)

In [8]:
ap.dtypes

index                                int64
merchant_id                         object
amount                             float64
type_request                        object
mvnt_id                             object
movement_type                       object
source_type                         object
created_at                  datetime64[ns]
eff_dt                      datetime64[ns]
report_date_local           datetime64[ns]
payment_date_local          datetime64[ns]
status                              object
file_name                           object
eff_dt_date                 datetime64[ns]
eff_dt_time                         object
created_at_date             datetime64[ns]
payment_date_local_prev     datetime64[ns]
report_id                           object
report_date_local_final     datetime64[ns]
payment_date_local_final    datetime64[ns]
status_final                        object
dtype: object

In [9]:
ap['source_type'].unique()

array(['DISB', 'SETT', 'RISK', 'PYMT', 'TOPUPS', 'LOAN', 'SETT_ADPTR',
       'TAXES'], dtype=object)

In [10]:
ap['report_id'].unique()

array(['AL87YCZ3P', 'A5GGCS72L', 'AFDCWJ6N2', ..., 'AY8KG295Z',
       'ABL9S383T', 'ADKRXTW8R'], dtype=object)

In [11]:
ap_sett = ap[(ap['source_type'] == 'SETT')]
ap_topups = ap[(ap['source_type'] == 'TOPUPS')]
ap_disb = ap[(ap['source_type'] == 'DISB')]
ap_loan = ap[(ap['source_type'] == 'LOAN')]
ap_pymt = ap[(ap['source_type'] == 'PYMT')]
ap_risk = ap[(ap['source_type'] == 'RISK')]
ap_sett_adptr = ap[(ap['source_type'] == 'SETT_ADPTR')]
ap_taxes = ap[(ap['source_type'] == 'TAXES')]
ap_refunds = ap[(ap['source_type'] == 'REFUNDS')]
ap_bill_pymnt = ap[(ap['source_type'] == 'BILL_PYMNT')]

In [12]:
pivot_prueba_1 = pd.pivot_table(ap,index=['report_id'],
                                columns=['movement_type'],
                                values=['amount'],aggfunc=['sum'],margins=True).reset_index()
pivot_prueba_1.fillna(0,inplace=True)
pivot_prueba_1.head()

report_id                 sum                                 \
                                      amount                                  
movement_type            ADJUSTMENT_PYMNT_RQ CANCEL_RQ CHRBCK_ADJUSTMENT_RQ   
0              A2223XG7Q                0.00      0.00                 0.00   
1              A2223Z6ZD                0.00      0.00                 0.00   
2              A2224B3CL                0.00      0.00                 0.00   
3              A2224F43G                0.00      0.00                 0.00   
4              A2224L4Z9                0.00      0.00                 0.00   

                                                                               \
                                                                                
movement_type CHRBCK_RQ COLLTN_RQ DEFERRED_REGULAR_PYMNT_RQ DIRECT_DEPOSIT_RQ   
0                  0.00      0.00                      0.00             24.00   
1                  0.00      0.00                      0.00              0.00   
2                  0.00      0.00                      0.00              0.00   
3                  0.00      0.00                      0.00              0.00   
4                  0.00      0.00                      0.00              0.00   

                                                                       \
                                                                        
movement_type   DISB_RQ DISPUTE_ADJUSTMENT_RQ DISPUTE_RQ FAST_PYMT_RQ   
0                -24.00                  0.00       0.00         0.00   
1               -871.04                  0.00       0.00       871.04   
2               -113.07                  0.00       0.00         0.00   
3             -7,141.28                  0.00       0.00         0.00   
4               -534.60                  0.00       0.00         0.00   

                                                                         \
                                                                          
movement_type IT_PYMT_RQ REGULAR_PYMT_RQ RELEASE_RQ RETURN_RQ REVIEW_RQ   
0                   0.00            0.00       0.00      0.00      0.00   
1                   0.00            0.00       0.00      0.00      0.00   
2                   0.00          113.07       0.00      0.00      0.00   
3                   0.00        7,141.28       0.00      0.00      0.00   
4                   0.00          534.60       0.00      0.00      0.00   

                                
                                
movement_type VAT_PYMT_RQ  All  
0                    0.00 0.00  
1                    0.00 0.00  
2                    0.00 0.00  
3                    0.00 0.00  
4                    0.00 0.00

In [13]:
pivot_prueba_1.shape

(1949686, 19)

In [14]:
pivot_prueba_1.columns = ['_'.join(col) for col in pivot_prueba_1.columns.values]
pivot_prueba_1.columns = pivot_prueba_1.columns.str.replace('sum_amount_','') 
pivot_prueba_1.rename(columns ={'report_id__':'report_id'}, inplace=True)
pivot_prueba_1.head()

,report_id,ADJUSTMENT_PYMNT_RQ,CANCEL_RQ,CHRBCK_ADJUSTMENT_RQ,CHRBCK_RQ,COLLTN_RQ,DEFERRED_REGULAR_PYMNT_RQ,DIRECT_DEPOSIT_RQ,DISB_RQ,DISPUTE_ADJUSTMENT_RQ,DISPUTE_RQ,FAST_PYMT_RQ,IT_PYMT_RQ,REGULAR_PYMT_RQ,RELEASE_RQ,RETURN_RQ,REVIEW_RQ,VAT_PYMT_RQ,All
0,A2223XG7Q,0.00,0.00,0.00,0.00,0.00,0.00,24.00,-24.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,A2223Z6ZD,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-871.04,0.00,0.00,871.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,A2224B3CL,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-113.07,0.00,0.00,0.00,0.00,113.07,0.00,0.00,0.00,0.00,0.00
3,A2224F43G,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"-7,141.28",0.00,0.00,0.00,0.00,"7,141.28",0.00,0.00,0.00,0.00,0.00
4,A2224L4Z9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-534.60,0.00,0.00,0.00,0.00,534.60,0.00,0.00,0.00,0.00,0.00


In [15]:
pivot_prueba_1.shape

(1949686, 19)

# DATA PROVISION

In [16]:
url = r'C:\Users\Cristian.Aparicio\Downloads\Ledger_final_marzo_2023.xlsx'

In [17]:
provision_file = pd.ExcelFile(url)
provision_file

In [18]:
provision_file.sheet_names

['Sheet1']

In [19]:
all_sheets = pd.read_excel(url, sheet_name=[0],
                           converters={'ORDER_ID':str,'LAST4':str,'AFFILIATION':str,'TRX_ID':str,
                                     'DEVICE_ID':str,'ULTIMOS_4':str})

In [20]:
all_sheets.keys()

dict_keys([0])

In [21]:
provision = pd.concat(all_sheets).reset_index()
provision.columns = provision.columns.str.strip().str.lower() 
provision.head()

,level_0,level_1,record_uuid,merchant_id,merchant_name,email,transaction_id,mvnt_type,pymt_prg,created_at,...,gross_amount,fees,taxes,net_amount,mdr,msi,number_msi,status_code,receipt_no,report_id
0,0,0,1300dae3-12a6-5c33-9e14-f112ce0fd15c,020a0ad3-4e44-426a-ad94-3b9800d457fd,MISCELANEA KARLA,ifer1482@hotmail.com,fb48a07c-c78d-40a2-afc7-cef04c86c4e6,STL_PCS_RQ,FAST_PAYMENT,2023-03-01T19:48:08Z,...,291.00,10.48,1.68,278.84,0.04,0.00,0,1,yYgZpvt,A4NY4JS42
1,0,1,57f40688-a60e-56b2-b822-8632138316e6,14ce31cb-7232-4c16-ba5b-b9f032402967,MINI SÃšPER ELI,eliazihs00@hotmail.com,7a016785-4a7d-4937-aa69-c69b4ded853e,STL_PCS_RQ,FAST_PAYMENT,2023-03-01T19:16:18Z,...,76.60,2.76,0.44,73.40,0.04,0.00,0,1,CqQlg3D,A8QS45JF8
2,0,2,c579e7dd-aabd-5b3a-a5b5-0b03ac0d2263,59c070f8-19ff-4e29-ba07-99f640e41007,ABARROTES MARINA,hugocalles087@gmail.com,39ba93a6-eb25-4b5e-b1de-08f0051f020b,STL_PCS_RQ,FAST_PAYMENT,2023-03-01T15:42:48Z,...,104.00,3.74,0.60,99.66,0.04,0.00,0,1,CwCkVsK,A4K2JB6W2
3,0,3,c393d00e-422f-5e96-b366-230220cbc79d,a4591f18-b475-4e63-bdaa-534657b12571,MISCELANEA SAMARATH,salvadorlsw@hotmail.com,22e49da3-f887-4a76-8a9f-5059674bae0d,STL_PCS_RQ,FAST_PAYMENT,2023-03-01T15:42:33Z,...,49.00,1.76,0.28,46.96,0.04,0.00,0,1,UWFaMWp,AFDQMWRVQ
4,0,4,aec54a7c-6e7e-5079-9c99-fe136a4fe902,6e3eeb6b-d284-4247-b52c-746c59395f17,ESTÃ‰TICA MAITE,mariateresajuareztorres@gmail.com,7276f489-3780-43c4-8ea0-ae9b28dcf210,STL_PCS_RQ,FAST_PAYMENT,2023-03-01T15:47:48Z,...,450.00,16.20,2.59,431.21,0.04,0.00,0,1,SkKsAsC,AMY89CF9B


In [22]:
provision.shape

(79236, 21)

In [23]:
list(pivot_prueba_1.columns)

['report_id',
 'ADJUSTMENT_PYMNT_RQ',
 'CANCEL_RQ',
 'CHRBCK_ADJUSTMENT_RQ',
 'CHRBCK_RQ',
 'COLLTN_RQ',
 'DEFERRED_REGULAR_PYMNT_RQ',
 'DIRECT_DEPOSIT_RQ',
 'DISB_RQ',
 'DISPUTE_ADJUSTMENT_RQ',
 'DISPUTE_RQ',
 'FAST_PYMT_RQ',
 'IT_PYMT_RQ',
 'REGULAR_PYMT_RQ',
 'RELEASE_RQ',
 'RETURN_RQ',
 'REVIEW_RQ',
 'VAT_PYMT_RQ',
 'All']

# CRUCE 2: DATA PROVISION VS AP JOURNAL (MOVEMENT TYPES)

In [24]:
cruce_2 = provision.merge(pivot_prueba_1[['report_id', 'ADJUSTMENT_PYMNT_RQ','CANCEL_RQ','CHRBCK_ADJUSTMENT_RQ','CHRBCK_RQ',
                                          'COLLTN_RQ','DEFERRED_REGULAR_PYMNT_RQ','DIRECT_DEPOSIT_RQ','DISB_RQ',
                                          'DISPUTE_ADJUSTMENT_RQ','DISPUTE_RQ','FAST_PYMT_RQ','IT_PYMT_RQ','REGULAR_PYMT_RQ',
                                          'RELEASE_RQ','RETURN_RQ','REVIEW_RQ','VAT_PYMT_RQ']],on='report_id',how='left')
cruce_2['ADJUSTMENT_PYMNT_RQ'].fillna('not_found',inplace=True)
cruce_2['CANCEL_RQ'].fillna('not_found',inplace=True)
cruce_2['CHRBCK_ADJUSTMENT_RQ'].fillna('not_found',inplace=True)
cruce_2['CHRBCK_RQ'].fillna('not_found',inplace=True)
cruce_2['COLLTN_RQ'].fillna('not_found',inplace=True)
cruce_2['DEFERRED_REGULAR_PYMNT_RQ'].fillna('not_found',inplace=True)
cruce_2['DIRECT_DEPOSIT_RQ'].fillna('not_found',inplace=True)
cruce_2['DISB_RQ'].fillna('not_found',inplace=True)
cruce_2['DISPUTE_ADJUSTMENT_RQ'].fillna('not_found',inplace=True)
cruce_2['DISPUTE_RQ'].fillna('not_found',inplace=True)
cruce_2['FAST_PYMT_RQ'].fillna('not_found',inplace=True)
cruce_2['IT_PYMT_RQ'].fillna('not_found',inplace=True)
cruce_2['REGULAR_PYMT_RQ'].fillna('not_found',inplace=True)
cruce_2['RELEASE_RQ'].fillna('not_found',inplace=True)
cruce_2['RETURN_RQ'].fillna('not_found',inplace=True)
cruce_2['REVIEW_RQ'].fillna('not_found',inplace=True)
cruce_2['VAT_PYMT_RQ'].fillna('not_found',inplace=True)
cruce_2.head()

,level_0,level_1,record_uuid,merchant_id,merchant_name,email,transaction_id,mvnt_type,pymt_prg,created_at,...,DISB_RQ,DISPUTE_ADJUSTMENT_RQ,DISPUTE_RQ,FAST_PYMT_RQ,IT_PYMT_RQ,REGULAR_PYMT_RQ,RELEASE_RQ,RETURN_RQ,REVIEW_RQ,VAT_PYMT_RQ
0,0,0,1300dae3-12a6-5c33-9e14-f112ce0fd15c,020a0ad3-4e44-426a-ad94-3b9800d457fd,MISCELANEA KARLA,ifer1482@hotmail.com,fb48a07c-c78d-40a2-afc7-cef04c86c4e6,STL_PCS_RQ,FAST_PAYMENT,2023-03-01T19:48:08Z,...,-780.47,0.00,0.00,780.47,0.00,0.00,0.00,0.00,0.00,0.00
1,0,1,57f40688-a60e-56b2-b822-8632138316e6,14ce31cb-7232-4c16-ba5b-b9f032402967,MINI SÃšPER ELI,eliazihs00@hotmail.com,7a016785-4a7d-4937-aa69-c69b4ded853e,STL_PCS_RQ,FAST_PAYMENT,2023-03-01T19:16:18Z,...,"-1,025.65",0.00,0.00,"1,025.65",0.00,0.00,0.00,0.00,0.00,0.00
2,0,2,c579e7dd-aabd-5b3a-a5b5-0b03ac0d2263,59c070f8-19ff-4e29-ba07-99f640e41007,ABARROTES MARINA,hugocalles087@gmail.com,39ba93a6-eb25-4b5e-b1de-08f0051f020b,STL_PCS_RQ,FAST_PAYMENT,2023-03-01T15:42:48Z,...,-99.66,0.00,0.00,99.66,0.00,0.00,0.00,0.00,0.00,0.00
3,0,3,c393d00e-422f-5e96-b366-230220cbc79d,a4591f18-b475-4e63-bdaa-534657b12571,MISCELANEA SAMARATH,salvadorlsw@hotmail.com,22e49da3-f887-4a76-8a9f-5059674bae0d,STL_PCS_RQ,FAST_PAYMENT,2023-03-01T15:42:33Z,...,-235.73,0.00,0.00,235.73,0.00,0.00,0.00,0.00,0.00,0.00
4,0,4,aec54a7c-6e7e-5079-9c99-fe136a4fe902,6e3eeb6b-d284-4247-b52c-746c59395f17,ESTÃ‰TICA MAITE,mariateresajuareztorres@gmail.com,7276f489-3780-43c4-8ea0-ae9b28dcf210,STL_PCS_RQ,FAST_PAYMENT,2023-03-01T15:47:48Z,...,-431.21,0.00,0.00,431.21,0.00,0.00,0.00,0.00,0.00,0.00


In [25]:
cruce_2.shape

(79236, 38)

# EXCEL FINAL

In [26]:
writer = pd.ExcelWriter('Data_only_marzo_ledger_2023 - ASIGNACION MOVIMIENTOS.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
cruce_2.to_excel(writer, sheet_name='Detalle', index=False)
writer.close()